## 🔐 Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI:

```bash
az login
```

# 🎯 Decorator Middleware

## Industry Use Case: Portfolio Rebalancing

This notebook demonstrates how to use `@agent_middleware` and `@function_middleware` **decorators** for cleaner middleware syntax.

| Feature | Benefit |
|---------|---------|
| **No Type Annotations** | Simpler, cleaner code |
| **Explicit Type** | Decorator declares middleware type |
| **Framework Handles Context** | Automatic context injection |

### FSI Scenario
A portfolio rebalancing agent with decorator-based middleware for:
- **Trading Window Check**: Ensure trades only execute during market hours
- **Position Logging**: Track all portfolio changes

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from root .env file
load_dotenv("../../.env", override=True)

PROJECT_ENDPOINT = os.getenv("AI_FOUNDRY_PROJECT_ENDPOINT")
MODEL_DEPLOYMENT = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME")

print(f"Project Endpoint: {PROJECT_ENDPOINT}")
print(f"Model Deployment: {MODEL_DEPLOYMENT}")

In [ ]:
import json
import time

from agent_framework import (
    agent_middleware,
    function_middleware,
)
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

print("✅ All imports loaded successfully")

## Define FSI Tool Functions

Portfolio rebalancing tools for investment management use case:

In [ ]:
def get_portfolio_allocation(account_id: str) -> str:
    """
    Get current portfolio allocation for an account.
    
    Args:
        account_id: The investment account identifier
        
    Returns:
        Current portfolio allocation details
    """
    # Simulated portfolio data
    portfolios = {
        "INV-001": {
            "account_id": "INV-001",
            "total_value": 500000,
            "allocations": {
                "US Equities": {"current": 45, "target": 40, "value": 225000},
                "International Equities": {"current": 20, "target": 25, "value": 100000},
                "Fixed Income": {"current": 25, "target": 25, "value": 125000},
                "Cash": {"current": 10, "target": 10, "value": 50000}
            },
            "risk_profile": "Moderate",
            "last_rebalanced": "2024-06-15"
        },
        "INV-002": {
            "account_id": "INV-002",
            "total_value": 1200000,
            "allocations": {
                "US Equities": {"current": 55, "target": 50, "value": 660000},
                "International Equities": {"current": 15, "target": 20, "value": 180000},
                "Fixed Income": {"current": 20, "target": 20, "value": 240000},
                "Alternatives": {"current": 5, "target": 5, "value": 60000},
                "Cash": {"current": 5, "target": 5, "value": 60000}
            },
            "risk_profile": "Aggressive",
            "last_rebalanced": "2024-05-01"
        }
    }
    
    if account_id not in portfolios:
        return json.dumps({"error": f"Account {account_id} not found"})
    
    return json.dumps(portfolios[account_id], indent=2)


def execute_rebalance(account_id: str, strategy: str = "proportional") -> str:
    """
    Execute portfolio rebalancing for an account.
    
    Args:
        account_id: The investment account identifier
        strategy: Rebalancing strategy ('proportional', 'threshold', 'tax_aware')
        
    Returns:
        Rebalancing execution result with trades
    """
    # Simulated rebalancing execution
    rebalance_results = {
        "INV-001": {
            "account_id": "INV-001",
            "strategy_used": strategy,
            "status": "completed",
            "trades_executed": [
                {"asset": "US Equities", "action": "SELL", "amount": 25000, "reason": "Over target by 5%"},
                {"asset": "International Equities", "action": "BUY", "amount": 25000, "reason": "Under target by 5%"}
            ],
            "estimated_tax_impact": 1250.00,
            "trading_costs": 75.00,
            "new_allocations": {
                "US Equities": 40,
                "International Equities": 25,
                "Fixed Income": 25,
                "Cash": 10
            },
            "execution_timestamp": "2024-12-20T10:30:00Z"
        },
        "INV-002": {
            "account_id": "INV-002",
            "strategy_used": strategy,
            "status": "completed",
            "trades_executed": [
                {"asset": "US Equities", "action": "SELL", "amount": 60000, "reason": "Over target by 5%"},
                {"asset": "International Equities", "action": "BUY", "amount": 60000, "reason": "Under target by 5%"}
            ],
            "estimated_tax_impact": 3000.00,
            "trading_costs": 180.00,
            "new_allocations": {
                "US Equities": 50,
                "International Equities": 20,
                "Fixed Income": 20,
                "Alternatives": 5,
                "Cash": 5
            },
            "execution_timestamp": "2024-12-20T10:30:00Z"
        }
    }
    
    if account_id not in rebalance_results:
        return json.dumps({"error": f"Account {account_id} not found for rebalancing"})
    
    result = rebalance_results[account_id]
    result["strategy_used"] = strategy
    return json.dumps(result, indent=2)

print("✅ FSI portfolio tools defined: get_portfolio_allocation, execute_rebalance")

## Decorator-Based Middleware

This sample demonstrates how to use `@agent_middleware` and `@function_middleware` decorators
to explicitly mark middleware functions **without requiring type annotations**.

### Industry Use Cases for Decorator Middleware

| Decorator | FSI Application |
|-----------|-----------------|
| `@agent_middleware` | Portfolio risk assessment before/after execution |
| `@function_middleware` | Trade authorization and compliance validation |

### Key Benefits of Decorator Approach

1. **Explicit middleware type declaration** - No ambiguity
2. **No type annotations needed** - Cleaner, simpler syntax  
3. **Framework handles context types** - Automatic type inference
4. **Prevents type mismatches** - Safety validation built-in

In [ ]:
# Portfolio Risk Assessment Middleware (Agent-level)
@agent_middleware  # Decorator marks this as agent middleware - no type annotations needed
async def portfolio_risk_middleware(context, next):  # type: ignore
    """Agent middleware that assesses portfolio risk before and after execution."""
    print("[Risk Assessment] Before agent execution")
    await next(context)
    print("[Risk Assessment] After agent execution")


# Trade Authorization Middleware (Function-level)
@function_middleware  # Decorator marks this as function middleware - no type annotations needed
async def trade_authorization_middleware(context, next):  # type: ignore
    """Function middleware that validates trade authorization before execution."""
    print(f"[Trade Auth] Before calling: {context.function.name}")  # type: ignore
    await next(context)
    print(f"[Trade Auth] After calling: {context.function.name}")  # type: ignore

print("✅ Decorator-based middleware defined:")
print("   - @agent_middleware: portfolio_risk_middleware")
print("   - @function_middleware: trade_authorization_middleware")

## Example: Portfolio Rebalancing with Decorator Middleware

Running the Portfolio Rebalancing Agent with decorator-based middleware stack:

In [ ]:
async def main():
    """Example demonstrating decorator-based middleware for portfolio rebalancing."""
    print("=== Decorator Middleware Example ===")
    
    # For authentication, run `az login` command in terminal
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(credential=credential).as_agent(
            name="PortfolioRebalancingAgent",
            instructions="You are a Portfolio Rebalancing Assistant. Call get_portfolio_allocation to check allocations and execute_rebalance to rebalance.",
            tools=[get_portfolio_allocation, execute_rebalance],
            middleware=[portfolio_risk_middleware, trade_authorization_middleware]
        ) as agent,
    ):
        query = "Check allocation for account INV-001 and rebalance it"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result.text if result.text else 'No response'}")

await main()

## 📋 Key Takeaways

### Decorator Middleware Benefits

| Feature | Description |
|---------|-------------|
| **Simple Syntax** | `@agent_middleware` and `@function_middleware` decorators |
| **No Type Annotations** | Framework automatically handles context types |
| **Explicit Intent** | Clear declaration of middleware type |
| **Safety Validation** | Prevents decorator/type annotation mismatches |

### FSI Applications

- **`@agent_middleware`**: Portfolio risk assessment, compliance logging, session management
- **`@function_middleware`**: Trade authorization, transaction validation, audit trails

### API Pattern Used

```python
# New decorator-based middleware pattern
@agent_middleware
async def my_middleware(context, next):
    # Pre-execution logic
    await next(context)
    # Post-execution logic

agent = AzureAIAgentClient(
    project_endpoint=PROJECT_ENDPOINT,
    AZURE_AI_MODEL_DEPLOYMENT_NAME=MODEL_DEPLOYMENT,
    credential=credential
).as_agent(
    name="MyAgent",
    middleware=[my_middleware]
)
```

